##### 【 의사결정나무 알고리즘 - 보팅】
- 특징 : 동일한 데이터셋 + 다양한 학습 알고리즘 모델들
- 결과
    * Hard Voting : 결론을 다수결로 결정
    * Soft Voting : 클래스별 확률값 평균 도출 후 최고 확률값의 클래스로 결정
    * 성능 : Hard Voting < Soft Voting
    * 주의 : Soft Voting 시 모델들은 predict_proba() 메서드 존재해야 함

[1] 모듈 로딩 및 데이터 준비<hr>

In [48]:
## ==================================================
## [1-1] 모듈 로딩
## ==================================================
import pandas as pd 
import numpy as np

## ML학습 관련
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

## ML 데이터셋 및 전처리 관련
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder

## ML CV, Pipeline 관련 => 모델 일반화/최적 하이퍼파라미터 조사 및 데이터 누수 해결
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline 

## ML 성능지표 관련
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import classification_report 

## 시각화 관련
import matplotlib.pyplot as plt
import graphviz

In [49]:
## ==================================================
## [1-2] 데이터 준비 및 확인
## ==================================================
## 데이터
DATA_FILE = '../Data/iris.csv'

## 데이터 로딩
irisDF = pd.read_csv(DATA_FILE)

## 데이터 기본정보 확인
display( irisDF.head(2) )
irisDF.info()


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


[2] 학습 준비<hr>

In [50]:
## ==================================================
## [2-1] 피쳐/타겟 분리
## ==================================================
featureDF = irisDF[irisDF.columns[:-1]]
targetSR  = irisDF[irisDF.columns[-1]]

print(f'featureDF:{featureDF.shape},  targetSR:{targetSR.shape}')

featureDF:(150, 4),  targetSR:(150,)


In [51]:
## ==================================================
## [2-2] 학습용/테스트용 분리
## ==================================================
x_train, x_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=targetSR)

print(f'[TRAIN] x_train:{x_train.shape},  y_train:{y_train.shape}')
print(f'[TEST] x_test:{x_test.shape},  y_test:{y_test.shape}')

[TRAIN] x_train:(120, 4),  y_train:(120,)
[TEST] x_test:(30, 4),  y_test:(30,)


In [52]:
## ==================================================
## [2-3] 타켓 컬럼 인코딩 처리 
## ==================================================
lbEncoder  = LabelEncoder()

en_y_train = lbEncoder.fit_transform(y_train)  ## 학습용 타겟으로 인코더 생성 후 변환까지 진행
en_y_test  = lbEncoder.transform(y_test)


In [53]:
## ==================================================
## [2-4] 수치형 피쳐 컬럼 스케일링 처리
## ==================================================
rbScaler  = RobustScaler()

rb_x_train = rbScaler.fit_transform(x_train)  ## 학습용 타겟으로 인코더 생성 후 변환까지 진행
rb_x_test  = rbScaler.transform(x_test)

[3] 학습 진행 - 단순 학습 <hr>

In [54]:
# 보팅 학습에 사용될 모델/학습기들 생성
knn = KNeighborsClassifier()
svc = SVC(probability=True)     # predict_proba() 메서드 활성화 설정\
dt = DecisionTreeClassifier(random_state=10)

In [55]:
# 보팅 인스턴스 생성
vtModel = VotingClassifier(estimators=[('knn', knn), ('svc', svc), ('dt', dt)])

# 보팅 동일 데이터셋으로 다른 학습 알고리즘으로 학습 진행
vtModel.fit(rb_x_train, en_y_train)

,estimators,"[('knn', ...), ('svc', ...), ...]"
,voting,'hard'
,weights,None
,n_jobs,None
,flatten_transform,True
,verbose,False
,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2


In [56]:
# 학습 후 모델 파라미터 확인
print(f"classes_    : {vtModel.classes_}")
print(f"estimators_ : {vtModel.estimators_}")
print(f"named_estimators_ : {vtModel.named_estimators_}")

classes_    : [0 1 2]
estimators_ : [KNeighborsClassifier(), SVC(probability=True), DecisionTreeClassifier(random_state=10)]
named_estimators_ : {'knn': KNeighborsClassifier(), 'svc': SVC(probability=True), 'dt': DecisionTreeClassifier(random_state=10)}


In [57]:
# 성능평가
vtModel.score(rb_x_test, en_y_test)

0.9666666666666667